# OR 확정모델 팀프로젝트 파이썬 코드 정리

* nodelst: node ID 저장해둔 리스트
* findlinkid: {(시작 노드, 끝 노드) = Link ID} 딕셔너리
* timedict: {Link ID : 걸리는 시간} 저장해둔 딕셔너리
* speeddict: {Link ID : 보정한 속도}
* freqdict: {Link ID: 어린이 보호구역 빈도}
* node_location: {node id: 그 노드의 위도, 경도,  이름}

##### 사용한 모듈
* import pandas
* import datetime
* import heqpq
* import folium
* import requests
* import json
* import math

##### 정의한 함수
* def gajung(linkid, t, x, y, z)
* def makegraph(t,x,y,z)
* def dijkstra(graph, start, end)
* def GET_END_POINT(q)
* def haversine(lon1, lat1, lon2, lat2)
* def get_start_time()
* def find_least_node()
* def drawmap(result)

In [1]:
import pandas as pd
import datetime

### csv 파일에서 데이터 불러와서 정제하기

In [2]:
b_datalstset = []
for i in range(1,5):
    b_datalstset.append(pd.read_csv("dataset\시간속도 - 평일 구간"+str(i)+".csv"))
    b_datalstset.append(pd.read_csv("dataset\시간속도 - 주말 구간"+str(i)+".csv"))
c_datalst = pd.read_csv("dataset\가중치c_t1t2.csv", encoding = 'cp949')
link_data = pd.read_csv("dataset\빈도계산_수정.csv",encoding = 'cp949')

In [3]:
node_data = pd.read_csv("dataset\구로구 표준노드.csv")
node_data = node_data[["NODE_ID","NODE_NAME","경도","위도"]]
node_data.dropna(inplace = True)
print(node_data.head())

# nodelst: node ID 저장해둔 리스트
nodelst = [int(i) for i in node_data.NODE_ID]
nodelst[:5]

        NODE_ID   NODE_NAME          경도         위도
0  1.160002e+09  만민중앙교회앞교차로  126.892491  37.481500
1  1.160002e+09  구로남초교입구교차로  126.893507  37.482330
2  1.160002e+09   디지털1단지교차로  126.895371  37.484558
3  1.160002e+09       천왕사거리  126.839829  37.484895
4  1.160003e+09     구로구-금천구  126.878755  37.486559


[1160001700, 1160002000, 1160002300, 1160002400, 1160002600]

In [4]:
# findlinkid: {(시작 노드, 끝 노드) = Link ID} 딕셔너리
findlinkid = {}
for i in range(link_data.count()[0]):
    ID = link_data.LINK_ID[i]
    start = link_data.F_NODE[i]
    end = link_data.T_NODE[i]
    findlinkid[(start,end)] = ID

### 가중치 계산하기
#### 가중치: T_ijt1t2 * (1+a_ij)^x * (1+b_ijt1t2)^y * (1+ c_t1t2)^z

In [5]:
# 나중에 t 입력받고 나서 가중치 함수 실행시켜서 채워주기
timedict = {}
freqdict = {}
speeddict = {}

In [6]:
def gajung(linkid, t, x, y, z):
    # timedict: {Link ID : 걸리는 시간} 저장해둔 딕셔너리
    for i,j in zip(b_datalstset[t].LINK_ID,b_datalstset[t].TIME):
        timedict[i]=j

    # freqdict: {Link ID: 어린이 보호구역 빈도}
    for link_id, frequency in zip(link_data.LINK_ID,link_data.빈도):
        freqdict[link_id] = frequency

    # speeddict: {Link ID : 보정한 속도}
    for i,j in zip(b_datalstset[t].LINK_ID, b_datalstset[t].SPEED):
        speeddict[i]=j
    
    # T_ijt1t2 = timedict[linkid]
    # a_ij = freqdict[linkid]
    # b_ij = speeddict[linkid]
    # c_t1t2 = c_datalst.c_t1t2[t]
    a = (freqdict[linkid] + 1) ** x
    b = (speeddict[linkid] + 1) ** y
    c = (c_datalst.c_t1t2[t] + 1) ** z
    gajungchi = timedict[linkid] * a * b * c
    
    return(round(gajungchi, 5))

### 구로구 데이터로 그래프 만들기

In [7]:
def makegraph(t,x,y,z):
    # 그래프의 각 간선에 대해 가중치 부여
    mygraph = {}
    for s_node in nodelst:
        mygraph[s_node] = {}

    for s_node, e_node in zip(link_data.F_NODE, link_data.T_NODE):
        mygraph[s_node][e_node] = gajung(findlinkid[(s_node,e_node)],t,x,y,z)
    
    return mygraph

### 다익스트라 함수 구현하기

In [8]:
import heapq

def dijkstra(graph, start, end):
    distances = {vertex: [float('inf'), start] for vertex in graph}
    distances[start] = [0, start] # 시작 지점과 시작 지점까지 거리 0으로 초기화

    queue = []
    heapq.heappush(queue, [distances[start][0], start])

    while queue:
        # 큐에서 정점을 하나씩 꺼내 인접한 정점들의 가중치를 모두 확인하여 업데이트합니다.
        current_distance, current_vertex = heapq.heappop(queue)
        
        # 더 짧은 경로가 있다면 무시한다.
        if distances[current_vertex][0] < current_distance:
            continue
            
        for adjacent, weight in graph[current_vertex].items():
            distance = current_distance + weight
            # 만약 시작 정점에서 인접 정점으로 바로 가는 것보다 현재 정점을 통해 가는 것이 더 가까울 경우에는
            if distance < distances[adjacent][0]:
                # 거리를 업데이트합니다.
                distances[adjacent] = [distance, current_vertex]
                heapq.heappush(queue, [distance, adjacent])
    
    path = end
    path_output = [end]
    while distances[path][1] != start:
        path_output.append(distances[path][1])
        path = distances[path][1]
    path_output.append(start)
    
    return distances[end][0], path_output[::-1]

### 네이버 지도에 주소값 검색해서 위도, 경도 찾아주기

https://github.com/GodMoonGoodman/NaverMap-Car-minimum-distance

위의 주소를 참고하여 구현하였습니다.

In [9]:
import requests as r
import json

In [10]:
http_header = {
    'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1',
    'x-requested-with': 'XMLHttpRequest'}
session = r.Session()
session.headers.update(http_header)

In [11]:
def GET_END_POINT(q):
    res = session.get('https://m.map.naver.com/apis/search/poi?query={}&page=1'.format(q)).text
    res_dict = json.loads(res)
    x = res_dict['result']['address']['list'][0]['x']
    y = res_dict['result']['address']['list'][0]['y']
    return float(x),float(y)

### 구면좌표계 거리 계산하기
https://www.it-swarm.dev/ko/python/%EB%91%90-%EC%A7%80%EC%A0%90-%EC%9C%84%EB%8F%84-%EA%B2%BD%EB%8F%84-%EC%82%AC%EC%9D%B4%EC%9D%98-%EA%B1%B0%EB%A6%AC%EB%A5%BC-%EC%8B%A0%EC%86%8D%ED%95%98%EA%B2%8C-%EC%B6%94%EC%A0%95%ED%95%98%EB%A0%A4%EB%A9%B4-%EC%96%B4%EB%96%BB%EA%B2%8C%ED%95%B4%EC%95%BC%ED%95%A9%EB%8B%88%EA%B9%8C/1072488907/

의 코드를 사용하였습니다.

In [12]:
from math import radians, cos, sin, asin, sqrt
# 위도: lat, 경도: lon
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km * 1000

### 출발 시각 입력받는 함수
* t: 출발 시각의 타입 (0~7의 8종류: 평일, 주말 각각 4가지 시간대)

In [13]:
def get_start_time():
    nowweekday = datetime.datetime.today().weekday()
    nowhour = datetime.datetime.now().strftime('%H')
    t1 = [1,1,1,1,1,1,1,1,2,2,1,1,2,2,3,3,3,3,4,4,1,1,1,1][int(nowhour)]
    t2 = [1,1,1,1,1,2,2][nowweekday]
    #dict_for_t = {(1,1):0, (2,1):1, (3,1):2, (4,1):3, (1,2):4, (2,2):5, (3,2):6, (4,2):7}
    t = t1 + t2 * 4 - 5
    #t = index of c_datalst : 어느 시간대에 속하나?
    
    print("현재 시각:", datetime.datetime.today())
    
    check = -1
    while True:
        try:
            check = input("지금 출발한다면 1을, 아니면 출발 예정 시각(ex:2020-06-12, 06:11 또는 오늘, 06:11)을 입력해주세요: ")
            if check == "1":
                return t
            if "오늘" in check:
                nowhour = int(check.split(',')[1].split(':')[0])
                t1 = [1,1,1,1,1,1,1,1,2,2,1,1,2,2,3,3,3,3,4,4,1,1,1,1][nowhour]
                return t1 + t2 * 4 - 5
            else:
                startday, starttime = check.split(',')
                nowhour = int(starttime.split(':')[0])
                t1 = [1,1,1,1,1,1,1,1,2,2,1,1,2,2,3,3,3,3,4,4,1,1,1,1][nowhour]
                
                year, month, day = startday.split('-')
                nowweekday = datetime.date(int(year),int(month),int(day)).weekday()
                t2 = [1,1,1,1,1,2,2][nowweekday]
                return t1 + t2 * 4 - 5
        except: pass
        

### 출발, 도착 주소 입력받아서 가장 가까운 노드 찾아주는 함수

In [14]:
# node_location: {node id: 그 노드의 위도, 경도,  이름}
node_location = {}
for i in range(node_data.count()[0]):
    node_location[int(node_data.NODE_ID[i])] = (node_data.위도[i], node_data.경도[i], node_data.NODE_NAME[i])

In [15]:
def find_least_node():
    while True:
        try:
            start_add = input("출발 주소를 입력해주세요: ")
            start_x, start_y = GET_END_POINT(start_add)
            break
        except:
            print("주소가 잘못되었습니다.")
    while True:
        try:
            end_add = input("도착 주소를 입력해주세요: ")
            end_x, end_y = GET_END_POINT(end_add)
            break
        except:
            print("주소가 잘못되었습니다.")
            
    # start_add = "서울특별시 구로구 궁동 230"
    # end_add = "서울특별시 구로구 고척1동 63-12"
    start = [0,10**10]
    end = [0,10**10]
    for i in node_location:
        tmp_y, tmp_x = node_location[i][:2]
        if start[1] > haversine(start_x,start_y,tmp_x,tmp_y):
            start[0] = i
            start[1] = haversine(start_x,start_y,tmp_x,tmp_y)
        if end[1] > haversine(end_x,end_y,tmp_x,tmp_y):
            end[0] = i
            end[1] = haversine(end_x,end_y,tmp_x,tmp_y)
    return start[0], end[0]

### 지도 그리기

In [16]:
import folium

In [17]:
protectzone = pd.read_csv("dataset\보호구역.csv", encoding = "cp949")

In [18]:
def drawmap(result):
    guromap = folium.Map(location=[37.496897,126.8594693], zoom_start=13)
    
    # 구로구 경계선 테두리와 내부 색칠하기
    guromap.choropleth(geo_data = "dataset\guroline.json", fill_color = "#EFFBF2",
                    fill_opacity = 0.4,
                   line_color = "#63C5DA",line_weight = 3)
    
    # 스쿨존 표시하기
    for i in range(protectzone.shape[0]):
        protect_a = protectzone.위도[i]
        protect_b = protectzone.경도[i]
        folium.CircleMarker([protect_a,protect_b],
                            color="#E8A317", fill="#E8A317",
                            opacity =  0.7).add_to(guromap)
        
    # 경로 선 따라서 그려주기
    for i,j in zip(result, result[1:]):
        locate = [list(node_location[i][0:2]), list(node_location[j][0:2])]
        folium.PolyLine(locations = locate, color = "#028A0F").add_to(guromap)
    
    # 시작지점과 도착지점에 마커 넣어주기
    i,j,k =  node_location[result[0]]
    folium.Marker([i,j], popup = "start", icon = folium.Icon(color="blue")).add_to(guromap)
    i,j,k =  node_location[result[-1]]
    folium.Marker([i,j], popup = "end", icon = folium.Icon(color="red")).add_to(guromap)
    
    
    
    
    return guromap

### 사용자 입력받는 부분

In [19]:
t = get_start_time()
start, end = find_least_node()

print("\n각각의 기피 정도에 대한 질문입니다. 0은 기피하지 않는다, 3이 가장 많이 기피한다입니다.")
x = y = z = -1
while x not in (0,1,2,3):
    x = int(input("x: 0,1,2,3중에 입력하세요: "))
while y not in (0,1,2,3):
    y = int(input("y: 0,1,2,3중에 입력하세요: "))
while z not in (0,1,2,3):
    z = int(input("z: 0,1,2,3중에 입력하세요: "))

mygraph = makegraph(t,x,y,z)
goal, result = dijkstra(mygraph, start, end)

time = sum([timedict[findlinkid[i]] for i in zip(result,result[1:])])
print("="*30 + "\n경로\n" + "="*30)
for i in result:
    print(node_location[i][2])
print("-"*30)
print("목적함수 z =",goal) # 목적함수의 최솟값
print("실제 걸리는 시간: {}분 {}초".format(time//60,time%60))

guromap = drawmap(result)
guromap

현재 시각: 2020-06-27 02:23:00.312595


지금 출발한다면 1을, 아니면 출발 예정 시각(ex:2020-06-12, 06:11 또는 오늘, 06:11)을 입력해주세요:  1
출발 주소를 입력해주세요:  서울특별시 구로구 궁동 230
도착 주소를 입력해주세요:  서울특별시 구로구 고척1동 63-12



각각의 기피 정도에 대한 질문입니다. 0은 기피하지 않는다, 3이 가장 많이 기피한다입니다.


x: 0,1,2,3중에 입력하세요:  3
y: 0,1,2,3중에 입력하세요:  1
z: 0,1,2,3중에 입력하세요:  2


경로
우신중고교입구교차로
수궁동사무소입구삼거
궁동3R
백조빌딩
수정아파트
온수입구삼거리
시티월드앞교차로
오류동교차로
오류동역교차로
미래빌라
오류동역2번출구
현대홈맨션
성은빌라
엑스포카센타
개봉고가차도
성한빌딩
고척동국제외국어학원
개봉사거리
금석빌딩앞
개봉동179-46
한마음아파트삼거리
영동빌딩
서울은행고척동지점
강서빌딩
서봉빌딩
윈골프클럽
구로소방서앞
------------------------------
목적함수 z = 15038.337459999999
실제 걸리는 시간: 32분 6초


C:\Users\sjkan\anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


출발 주소 예시 : 서울특별시 구로구 궁동 230

도착 주소 예시 : 서울특별시 구로구 고척1동 63-12

### OPL과의 Computaion time 비교

In [20]:
import time

In [21]:
node_connect = [[0] * 452 for _ in range(452)]

In [22]:
now = time.time()
mygraph = makegraph(0,3,1,2)
start,end = (1160016800, 1160026900)
goal, result = dijkstra(mygraph, start, end)
print("목적함수 z =",goal)
for i,j in zip(result,result[1:]):
    node_connect[nodelst.index(i)][nodelst.index(j)] = 1

for i in node_connect:
    pass

print(time.time()-now)

목적함수 z = 10495.289509999999
5.0531041622161865
